### Zigbang 원룸 매물 데이터 수집


In [46]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import requests, json
pd.options.display.max_columns = 40

#### Process

    - 동이름으로 위도 경도 구하기
    - 위도 경도로 geohash 알아내기
    - geohash로 매물 아이디 가져오기
    - 매물 아이디로 매물 정보 가져오기


#### 1. 동이름으로 위도 경도 구하기


In [109]:
# 1. 웹페이지분석 : 개발자도구 : URL
addr = '회현동'
url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'

# 2. Request(URL) >Response
response = requests.get(url)
response

# 3. JSON > data[위도, 경도]
data = response.json()["items"][0]
lat, lng = data["lat"], data["lng"]
lat, lng

(37.52959077840698, 126.89981785641798)

#### 2. 위도 경도로 geohash 알아내기

- install geohash2 package


In [110]:
import geohash2

In [111]:
# precision : 영역크기 : 커질수록 영역이 작아짐
geohash = geohash2.encode(lat, lng, precision=5)
geohash

'wydjx'

#### 3. geohash로 매물 아이디 가져오기


In [112]:
url = 'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash=wydjx&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
    
response = requests.get(url)
response

<Response [200]>

In [114]:
items = response.json()['items']
ids = []
for item in items:
    ids.append(item['item_id'])
len(ids), ids[:5]

(580, [37623131, 37685329, 37699264, 37828417, 37595863])

In [115]:
# listcomprehension : 간단한 for, if를 사용해서 리스트 데이터 만들때 사용되는 문법
# [var for var in collection if condition]
ids = [item['item_id'] for item in items]
ids[:5]

[37623131, 37685329, 37699264, 37828417, 37595863]

#### 4. 매물 아이디로 매물 정보 가져오기


In [116]:
# 요청 id가 1000를 초과하면 에러 발생
url = 'https://apis.zigbang.com/v2/items/list'
params = {
    'domain' : 'zigbang', 'item_ids' : ids
}
response = requests.post(url, params)

In [117]:
data = response.json()['items']
df = pd.DataFrame(data)


columns = ['item_id','sales_type','deposit','floor', 'floor_string', 'building_floor', 'title','manage_cost','service_type','size_m2','address1']

df = df[columns]
df.tail(2)

,item_id,sales_type,deposit,floor,floor_string,building_floor,title,manage_cost,service_type,size_m2,address1
578,37703323,월세,5000,3,3,6,전입o 가좌역 도보5분 살기좋은 신축쓰리룸,8,빌라,63.97,서울시 마포구 중동
579,37836529,월세,5000,고,고,6,가좌역 역세권 신축 첫입주 투룸 반전세주차O,8,오피스텔,61.09,서울시 마포구 중동


In [118]:
망원_df = df[df['address1'].str.contains('망원동')]
망원_df.tail(2)

,item_id,sales_type,deposit,floor,floor_string,building_floor,title,manage_cost,service_type,size_m2,address1
501,37804362,월세,300,2,2,3,신축급즉시입주깔끔원룸 주차가능,7,원룸,19.83,서울시 마포구 망원동
502,37811722,월세,2000,1,1,3,리모델링후 첫입주 쓰리룸월세 풀옵션,1,빌라,48.00,서울시 마포구 망원동


In [123]:
def oneroom(addr):
    url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
    response = requests.get(url)
    data = response.json()["items"][0]
    lat, lng = data["lat"], data["lng"]
    geohash = geohash2.encode(lat, lng, precision=5)
    
    url = 'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash=wydjx&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
    response = requests.get(url)
    items = response.json()['items']
    ids = [item['item_id'] for item in items]
    
    url = 'https://apis.zigbang.com/v2/items/list'
    params = {'domain' : 'zigbang', 'item_ids' : ids}
    response = requests.post(url, params)
    data = response.json()['items']
    df = pd.DataFrame(data)
    result = df[df['address1'].str.contains(addr)].reset_index(drop=True)
    
    return result[['item_id','sales_type','deposit','floor', 'floor_string', 'building_floor', 'title','manage_cost','service_type','size_m2','address1']]


In [127]:
res = oneroom('중동')
res

,item_id,sales_type,deposit,floor,floor_string,building_floor,title,manage_cost,service_type,size_m2,address1
0,37751115,월세,2000,1,1,2,올리모델링 가좌역5분 역세권 귀한 투룸ㅣ빠른입주OK,0,빌라,37.98,서울시 마포구 중동
1,37703323,월세,5000,3,3,6,전입o 가좌역 도보5분 살기좋은 신축쓰리룸,8,빌라,63.97,서울시 마포구 중동
2,37836529,월세,5000,고,고,6,가좌역 역세권 신축 첫입주 투룸 반전세주차O,8,오피스텔,61.09,서울시 마포구 중동


In [73]:
# 모듈 파일 만들기 : 변수, 함수, 클리스를 하나의 파일(.py)로 묶어서 코드 작성 실행 문법

In [83]:
def oneroom(addr):
    # url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={a/ddr}&serviceType=원룸'
    url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash=wydm8&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸q={addr}'
    
    response = requests.get(url)
    data = response.json()["items"][0]
    lat, lng = data["lat"], data["lng"]
    geohash = geohash2.encode(lat, lng, precision=5)
    
    url = 'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash=wydjx&needHasNoFiltered=true\
        &rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
    response = requests.get(url)
    items = response.json()['items']
    ids = [item['item_id'] for item in items]
    
    url = 'https://apis.zigbang.com/v2/items/list'
    params = {'domain' : 'zigbang', 'item_ids' : ids}
    response = requests.post(url, params)
    data = response.json()['items']
    df = pd.DataFrame(data)
    result = df[df['address1'].str.contains(addr)].reset_index(drop=True)
    
    return result[['item_id','sales_type','deposit','floor', 'floor_string', 'building_floor', 'title','manage_cost','service_type','size_m2','address1']]


In [82]:
import zigbang as zb

In [80]:
df = zb.oneroom('망원동')

KeyError: 'items'